https://github.com/harryhan618/SCNN_Pytorch

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
import jdc

In [ ]:
class SCNN(nn.Module):
  def __init__(self, input_size, ms_ks=9, pretrained=True):
    """
    Argument
      ms_ks: kernel size in message passing conv
    """
    super(SCNN, self).__init__()
    self.pretrained = pretrained
    self.net_init(input_size, ms_ks)
    if not pretrained:
      self.weight_init()

    self.scale_background = 0.4
    self.scale_seg = 1.0
    self.scale_exist = 0.1

    self.ce_loss = nn.CrossEntropyLoss(weight=torch.tensor([self.scale_background, 1, 1, 1, 1]))
    self.bce_loss = nn.BCELoss()

In [ ]:
%%add_to SCNN
def weight_init(self):
  for m in self.modules():
    if isinstance(m, nn.Conv2d):
      m.reset_parameters()
    elif isinstance(m, nn.BatchNorm2d):
      m.weight.data[:] = 1.
      m.bias.data.zero_()

In [ ]:
%%add_to SCNN
def net_init(self, input_size, ms_ks):
  input_w, input_h = input_size
  self.fc_input_feature = 5 * int(input_w/16) * int(input_h/16)
  self.backbone = models.vgg16_bn(pretrained=self.pretrained).features

  # ----------------- process backbone -----------------
  for i in [34, 37, 40]:
    conv = self.backbone._modules[str(i)]
    dilated_conv = nn.Conv2d(
        conv.in_channels, conv.out_channels, conv.kernel_size, stride=conv.stride,
        padding=tuple(p * 2 for p in conv.padding), dilation=2, bias=(conv.bias is not None)
    )
    dilated_conv.load_state_dict(conv.state_dict())
    self.backbone._modules[str(i)] = dilated_conv
  self.backbone._modules.pop('33')
  self.backbone._modules.pop('43')

  # ----------------- SCNN part -----------------
  self.layer1 = nn.Sequential(
    nn.Conv2d(512, 1024, 3, padding=4, dilation=4, bias=False),
    nn.BatchNorm2d(1024),
    nn.ReLU(),
    nn.Conv2d(1024, 128, 1, bias=False),
    nn.BatchNorm2d(128),
    nn.ReLU()  # (nB, 128, 36, 100)
  )

  # ----------------- add message passing -----------------
  self.message_passing = nn.ModuleList()
  self.message_passing.add_module('up_down', nn.Conv2d(128, 128, (1, ms_ks), padding=(0, ms_ks // 2), bias=False))
  self.message_passing.add_module('down_up', nn.Conv2d(128, 128, (1, ms_ks), padding=(0, ms_ks // 2), bias=False))
  self.message_passing.add_module('left_right', nn.Conv2d(128, 128, (ms_ks, 1), padding=(ms_ks // 2, 0), bias=False))
  self.message_passing.add_module('right_left', nn.Conv2d(128, 128, (ms_ks, 1), padding=(ms_ks // 2, 0), bias=False))
  # (nB, 128, 36, 100)

  # ----------------- SCNN part -----------------
  self.layer2 = nn.Sequential(
    nn.Dropout2d(0.1),
    nn.Conv2d(128, 5, 1)  # get (nB, 5, 36, 100)
  )

  self.layer3 = nn.Sequential(
    nn.Softmax(dim=1),  # (nB, 5, 36, 100)
    nn.AvgPool2d(2, 2),  # (nB, 5, 18, 50)
  )
  self.fc = nn.Sequential(
    nn.Linear(self.fc_input_feature, 128),
    nn.ReLU(),
    nn.Linear(128, 4),
    nn.Sigmoid()
  )

In [ ]:
%%add_to SCNN
def forward(self, img, seg_gt=None, exist_gt=None):
  x = self.backbone(img)
  x = self.layer1(x)
  x = self.message_passing_forward(x)
  x = self.layer2(x)

  seg_pred = F.interpolate(x, scale_factor=8, mode='bilinear', align_corners=True)
  x = self.layer3(x)
  x = x.view(-1, self.fc_input_feature)
  exist_pred = self.fc(x)

  if seg_gt is not None and exist_gt is not None:
    loss_seg = self.ce_loss(seg_pred, seg_gt)
    loss_exist = self.bce_loss(exist_pred, exist_gt)
    loss = loss_seg * self.scale_seg + loss_exist * self.scale_exist
  else:
    loss_seg = torch.tensor(0, dtype=img.dtype, device=img.device)
    loss_exist = torch.tensor(0, dtype=img.dtype, device=img.device)
    loss = torch.tensor(0, dtype=img.dtype, device=img.device)
  return seg_pred, exist_pred, loss_seg, loss_exist, loss


In [ ]:
%%add_to SCNN
def message_passing_forward(self, x):
  Vertical = [True, True, False, False]
  Reverse = [False, True, False, True]
  for ms_conv, v, r in zip(self.message_passing, Vertical, Reverse):
    x = self.message_passing_once(x, ms_conv, v, r)
  return x

In [ ]:
%%add_to SCNN
def message_passing_once(self, x, conv, vertical=True, reverse=False):
  """
  Argument:
  ----------
  x: input tensor
  vertical: vertical message passing or horizontal
  reverse: False for up-down or left-right, True for down-up or right-left
  """
  nB, C, H, W = x.shape
  if vertical:
    slices = [x[:, :, i:(i + 1), :] for i in range(H)]
    dim = 2
  else:
    slices = [x[:, :, :, i:(i + 1)] for i in range(W)]
    dim = 3
  if reverse:
    slices = slices[::-1]

  out = [slices[0]]
  for i in range(1, len(slices)):
    out.append(slices[i] + F.relu(conv(out[i - 1])))
  if reverse:
    out = out[::-1]
  return torch.cat(out, dim=dim)

In [ ]:
scn = SCNN((224,224), pretrained=False)

In [ ]:
scn

SCNN(
  (backbone): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 25